# Convolutional 2D VAE

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")

# %cd drive/MyDrive/Colab\ Notebooks/

Import necessary packages

In [ ]:
from glob import glob
import numpy as np
from scipy.spatial.distance import pdist, squareform
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import linalg as la
from keras import regularizers
from keras import backend as K
from keras.layers import (
    Conv2D,
    Conv2DTranspose,
    Input,
    Flatten,
    Dense,
    Lambda,
    Reshape,
)

## Data Input and Pre-Processing

Define core features of the dataset.

In [ ]:
dim = 2
numpart = 30
latent_dim = 40
box_size = 10

Import and reshape data.

In [ ]:
train_files = np.asarray(glob("gamma*_100000_*_x.txt"), dtype="str")
test_files = np.asarray(glob("gamma*_10000_*_x.txt"), dtype="str")

if train_files.size != test_files.size:
    print("Missing files!")

gamma = np.sort([f.split("_")[1] for f in train_files]).astype(float)
num_gammas = train_files.size

for files in train_files, test_files:
    files = files[np.argsort([f.split("_")[1] for f in files])]

train_arrays = [np.loadtxt(f) for f in train_files]
train_data = np.vstack(train_arrays)
train_data = train_data.reshape((-1, numpart, dim)) / (box_size * np.sqrt(dim))
train_labels = np.hstack([[i] * len(a) for i, a in enumerate(train_arrays)])

test_arrays = [np.loadtxt(f) for f in test_files]
test_data = np.vstack(test_arrays)
test_data = test_data.reshape((-1, numpart, dim)) / (box_size * np.sqrt(dim))
test_labels = np.hstack([[i] * len(a) for i, a in enumerate(test_arrays)])

print(f"Training data: {train_data.shape}\nTest data: {test_data.shape}")

Training data: (400000, 30, 2)
Test data: (40000, 30, 2)


Sort by distance from origin

In [ ]:
sort_idx = np.argsort(train_data[:, :, 0]**2 + train_data[:, :, 1]**2)
train_data = np.array(
    [sample[sort_idx[i]] for i, sample in enumerate(train_data)]
)

sort_idx = np.argsort(test_data[:, :, 0]**2 + test_data[:, :, 1]**2)
test_data = np.array(
    [sample[sort_idx[i]] for i, sample in enumerate(test_data)]
)

### Compute distance matrices

In [ ]:
# metric="euclidean", force="no", checks=True are by default
train_dm = np.array([squareform(pdist(sample)) for sample in train_data])
test_dm = np.array([squareform(pdist(sample)) for sample in test_data])

print(f"Training distance matrix shape: {train_dm.shape}")
print(f"Test distance matrix shape: {test_dm.shape}")


Training distance matrix shape: (400000, 30, 30)
Test distance matrix shape: (40000, 30, 30)


In [ ]:
m_train = train_data.shape[0]
m_test = test_data.shape[0]

train_perm = np.random.permutation(m_train)
test_perm = np.random.permutation(m_test)

train_data = train_data[train_perm]
train_dm = train_dm[train_perm]
train_labels = train_labels[train_perm]

test_data = test_data[test_perm]
test_dm = test_dm[test_perm]
test_labels = test_labels[test_perm]


## Variational Auto Encoder (Model 1)

### Sampling class

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### Encoder

In [ ]:
encoder_inputs = keras.Input(shape=(numpart, numpart, 1))
x = Conv2D(60, 3, padding="same", activation="relu")(encoder_inputs)
x = Conv2D(50, 3, padding="same", activation="relu")(x)
conv_shape = K.int_shape(x)  # Shape of conv to be provided to decoder
x = Flatten()(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(
    encoder_inputs, [z_mean, z_log_var, z], name="encoder"
)
encoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 30, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 30, 30, 60)   600         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 30, 30, 50)   27050       ['conv2d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 45000)        0           ['conv2d_1[0][0]']               
                                                                                            

### Decoder

In [ ]:
decoder_input = Input(shape=(latent_dim,), name="decoder_input")
x = Dense(
    conv_shape[1] * conv_shape[2] * conv_shape[3], activation="relu"
)(decoder_input)
x = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
x = Conv2DTranspose(50, 3, padding="same", activation="relu")(x)
x = Conv2DTranspose(60, 3, padding="same", activation="relu")(x)
decoder_outputs = Conv2DTranspose(
    1, 3, padding="same", activation="sigmoid", name="decoder_output"
)(x)

decoder = keras.Model(decoder_input, decoder_outputs, name="decoder")
decoder.summary()


Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 40)]              0         
                                                                 
 dense (Dense)               (None, 45000)             1845000   
                                                                 
 reshape (Reshape)           (None, 30, 30, 50)        0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 30, 30, 50)       22550     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 30, 30, 60)       27060     
 ranspose)                                                       
                                                                 
 decoder_output (Conv2DTrans  (None, 30, 30, 1)        541 

### VAE Class

In [ ]:
reg_lambda = 0.00035


class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            # Extract dimensions excluding the first 'None' dimension
            size = reconstruction.shape[1:]
            # noise = np.random.normal(0, 0.1, size=size)
            # reconstruction = reconstruction + noise

            # Reshape data to match decoder output shape
            data = tf.expand_dims(data, axis=-1)

            reconstruction_loss = tf.reduce_mean(
                keras.losses.mean_squared_error(data, reconstruction)
            )
            kl_loss = -0.5 * (
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            )
            kl_loss = tf.reduce_mean(kl_loss)
            total_loss = reconstruction_loss + reg_lambda * kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


### Train VAE

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001)
)  # lower learning rate
fit = vae.fit(train_dm, epochs=100, batch_size=128, verbose=2)


Epoch 1/100
3125/3125 - 81s - loss: 0.0036 - reconstruction_loss: 0.0030 - kl_loss: 1.8186 - 81s/epoch - 26ms/step
Epoch 2/100
3125/3125 - 70s - loss: 0.0020 - reconstruction_loss: 0.0012 - kl_loss: 2.0394 - 70s/epoch - 22ms/step
Epoch 3/100
3125/3125 - 70s - loss: 0.0018 - reconstruction_loss: 0.0011 - kl_loss: 2.0246 - 70s/epoch - 23ms/step
Epoch 4/100
3125/3125 - 71s - loss: 0.0017 - reconstruction_loss: 0.0010 - kl_loss: 2.0137 - 71s/epoch - 23ms/step
Epoch 5/100
3125/3125 - 72s - loss: 0.0016 - reconstruction_loss: 9.4657e-04 - kl_loss: 2.0053 - 72s/epoch - 23ms/step
Epoch 6/100
3125/3125 - 72s - loss: 0.0016 - reconstruction_loss: 9.0504e-04 - kl_loss: 1.9999 - 72s/epoch - 23ms/step
Epoch 7/100
3125/3125 - 72s - loss: 0.0016 - reconstruction_loss: 8.7281e-04 - kl_loss: 1.9942 - 72s/epoch - 23ms/step
Epoch 8/100
3125/3125 - 72s - loss: 0.0015 - reconstruction_loss: 8.4728e-04 - kl_loss: 1.9915 - 72s/epoch - 23ms/step
Epoch 9/100


In [ ]:
plt.rcParams["font.size"] = 12
fig, AX = plt.subplots(1, 2, figsize=(14, 6.0))
ax = AX[0]
ax.plot(fit.history["loss"], label="MSE loss", c="b")
ax.set_xlabel("Epoch")
ax.set_ylabel("MSE loss")
ax.legend()
ax = AX[1]
ax.plot(fit.history["kl_loss"], label="KL loss", c="r")
ax.set_xlabel("Epoch")
ax.set_ylabel("KL loss")
ax.legend()


In [ ]:
from google.colab import files

%rm -rf saved_model
%mkdir -p saved_model
tf.keras.models.save_model(vae.encoder, "saved_model/encoder")
tf.keras.models.save_model(vae.decoder, "saved_model/decoder")
!zip -r encoder.zip saved_model/encoder
!zip -r decoder.zip saved_model/decoder

files.download("encoder.zip")
files.download("decoder.zip")

## Evaluate performance
We'll now use the test set to explore the latent space distribution of data and the reconstruction accuracy

In [ ]:
encoded_test = np.array(vae.encoder.predict(test_dm))
encoded_train = np.array(vae.encoder.predict(train_dm))

print(encoded_test.shape)

We can now use the data to decode

In [ ]:
decoded_test = np.array(decoder.predict(encoded_test[2, :, :])).reshape(
    -1, numpart, numpart
)
decoded_train = np.array(decoder.predict(encoded_train[2, :, :])).reshape(
    -1, numpart, numpart
)
print(decoded_test.shape)


### Check reconstruction

In [ ]:
ind = 42
df = pd.DataFrame(decoded_test[ind])
sns.heatmap(data=df)


In [ ]:
df2 = pd.DataFrame(test_dm[ind])
sns.heatmap(data=df2)


In [ ]:
df3 = pd.DataFrame(abs(test_dm[ind] - decoded_test[ind]))
sns.heatmap(data=df3, cmap="mako")


## Alternative coordinates reconstructor
Another way to achieve coordinates from the distance matrix is through a more analytical method that does not involve Machine Learning

Step 1: retrieve coordinates from distance matrix (shifted and rotated)

In [ ]:
def gram_to_coordinates(distance_matrix):
    # Get the number of points
    n = distance_matrix.shape[0]

    # Compute the Gram matrix
    gram_matrix = -0.5 * (distance_matrix**2)

    # Center the Gram matrix
    gram_matrix_centered = (
        gram_matrix
        - np.mean(gram_matrix, axis=0)
        - np.mean(gram_matrix, axis=1)[:, np.newaxis]
        + np.mean(gram_matrix)
    )

    # Perform eigendecomposition of the centered Gram matrix
    eigenvalues, eigenvectors = np.linalg.eigh(gram_matrix_centered)

    # Sort eigenvalues and eigenvectors in descending order
    indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[indices]
    eigenvectors = eigenvectors[:, indices]

    # Extract the positive square root of eigenvalues
    sqrt_eigenvalues = np.sqrt(np.maximum(eigenvalues, 0))

    # Compute the coordinates of the points in 2D space
    coordinates = eigenvectors[:, :2] * sqrt_eigenvalues[:2]

    return coordinates

In [ ]:
coordinates = gram_to_coordinates(decoded_train[1])
print("Coordinates of points:")
print(coordinates)


In [ ]:
ind = 1
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
l = np.sqrt(2)
ax.scatter(
    coordinates[:, 0] * l,
    coordinates[:, 1] * l,
    s=30,
    c="#e63946",
)
ax.scatter(
    test_data[ind, :, 0] * l,
    test_data[ind, :, 1] * l,
    s=30,
    c="#023e8a",
)

In [ ]:
def align_points(points1, points2):
    # Center the points by subtracting their means
    centered_points1 = points1 - np.mean(points1, axis=0)
    centered_points2 = points2 - np.mean(points2, axis=0)

    # Compute the covariance matrix
    covariance_matrix = centered_points2.T @ centered_points1

    # Perform singular value decomposition (SVD)
    U, _, Vt = np.linalg.svd(covariance_matrix)

    # Calculate the optimal rotation matrix
    rotation_matrix = Vt.T @ U.T

    # Calculate the optimal translation vector
    translation_vector = np.mean(points2, axis=0) - np.mean(
        points1 @ rotation_matrix, axis=0
    )

    # Transform points1 using the estimated rotation and translation
    transformed_points = points1 @ rotation_matrix + translation_vector

    return transformed_points

In [ ]:
coordinates = align_points(coordinates, train_data[1])
ind = 1
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
l = np.sqrt(2)
ax.scatter(
    coordinates[:, 0] * l,
    coordinates[:, 1] * l,
    s=30,
    c="#e63946",
)
ax.scatter(
    train_data[ind, :, 0] * l,
    train_data[ind, :, 1] * l,
    s=30,
    c="#023e8a",
)

In [ ]:
rec_test_dec = np.zeros((test_data.shape))

for i in range(0, len(rec_test_dec)):
    rec_test_dec[i] = align_points(
        gram_to_coordinates(decoded_test[i]), test_data[i]
    )

In [ ]:
ind = 70
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
l = np.sqrt(2)
ax.scatter(
    rec_test_dec[ind, :, 0] * l,
    rec_test_dec[ind, :, 1] * l,
    s=30,
    c="#e63946",
)
ax.scatter(
    test_data[ind, :, 0] * l,
    test_data[ind, :, 1] * l,
    s=30,
    c="#023e8a",
)


## Deez Labels

In [ ]:
import sklearn
from sklearn.decomposition import PCA


def label_vis(vae, data, labels):
    # prediction
    z_mean, _, _ = vae.encoder.predict(data)

    pca = PCA(n_components=2)
    transformed_data = pca.fit_transform(z_mean)
    variance_ratio = pca.explained_variance_ratio_
    print(variance_ratio)

    # plot
    plt.figure(figsize=(5, 5))
    plt.scatter(transformed_data[:, 0], transformed_data[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()


In [ ]:
label_vis(vae, train_data, train_labels)

## Energy test

In [ ]:
def potential(x, gamma):
    """
    Calculate LJ + gravitational potential given a positions array.

    INPUTS
        x: array of shape (num_particles, dimension)
        gamma
    """

    n = len(x)
    pot = 0.0
    for i in range(n - 1):
        pot += gamma * x[i, -1]
        for j in range(i + 1, n):
            r2 = np.sum((x[i, :] - x[j, :]) ** 2)
            if r2 < 1.0:
                continue
            if r2 < 9.0:  # r_cut = 3 sigma
                sr6 = 1.0 / r2**3
                pot += 4 * (sr6**2 - sr6)
    pot += gamma * x[-1, -1]

    return pot

In [ ]:
scale_factor = box_size * np.sqrt(dim)
ori_pots = [
    potential(sample, gamma[test_labels[idx]])
    for idx, sample in enumerate(testset_conf * scale_factor)
]
rec_pots = [
    potential(sample, gamma[test_labels[idx]])
    for idx, sample in enumerate(rec_test_dec * scale_factor)
]


In [ ]:
nbins = 100

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ori_n, ori_bins, _ = ax[0].hist(ori_pots, bins=nbins, facecolor="gold")
ax[0].set_title("Original")
rec_n, rec_bins, _ = ax[1].hist(rec_pots, bins=nbins, facecolor="blue")
ax[1].set_title("Reconstruction")


for i in range(2):
    ax[i].set_xlabel("Potential")
    ax[i].set_ylabel("Counts per bin")
    ax[i].grid(axis="y", alpha=0.1, color="black")


Now we will compare the two distributions by calculating their Jensen–Shannon divergence, a symmetrized version of the Kullback–Leibler divergence, defined as

$$
    \operatorname{JSD}(P \:\Vert\: Q) = \frac{1}{2} D(P \:\Vert\: M)
        + \frac{1}{2} D(Q \:\Vert\: M),
$$

where $D$ is the usual Kullback–Leibler and $M = (P + Q) / 2$. $P$ and $Q$ must be normalized probability density functions: in our case they will be the bin heights of the previous histograms.

In [ ]:
from scipy.spatial.distance import jensenshannon


def jsd(p, q):
    """
    Returns the Jensen-Shannon divergence of two datasets.
    """

    p = np.asarray(p, dtype="float")
    q = np.asarray(q, dtype="float")

    # get optimal bin edges ("fd" tries to minimize the integral
    # of the squared difference between the histogram and
    # the theoretical pdf)
    rng_pq = (min(np.min(p), np.min(q)), max(np.max(p), np.max(q)))
    dxp = np.histogram_bin_edges(p, bins="fd", range=rng_pq)
    dxq = np.histogram_bin_edges(q, bins="fd", range=rng_pq)

    nb = max(len(dxp), len(dxq))
    dx = min(np.diff(dxp)[0], np.diff(dxq)[0])

    p, _ = np.histogram(p, bins=nb, range=rng_pq)
    q, _ = np.histogram(q, bins=nb, range=rng_pq)

    # jensenshannon from scipy normalizes p, q automatically
    return jensenshannon(p, q)


In [ ]:
jsd(ori_pots, rec_pots)
